# Demo


### Setup


In [ ]:
import numpy as np

from histogram_filter.world import Color
from histogram_filter.utils import propagate, sense, setup, visualize_belief, format_belief


### Random Walk


In [ ]:
my_env = setup()

num_steps = 10
for i in range(num_steps):
    if i > 0:
        my_env.move()
    print(f"Step: {i}")
    print(my_env)
    print(f"Sense: {my_env.sense().name}", "\n")


### 1D Histogram Filter, without world interaction


In [ ]:
my_env = setup()
belief = np.ones(len(my_env.grid)) / len(my_env.grid)
transition_matrix = my_env.get_transition_matrix()
sensor_matrix = my_env.get_sensor_matrix()
measurement_vector = [Color.ORANGE, Color.BLUE, Color.ORANGE]

for i, measurement in enumerate(measurement_vector):
    if i == 0:
        print("Grid:")
        print(my_env.grid, "\n")
        print("Initial belief:")
        print(format_belief(belief), "\n")

    print(f"Step: {i + 1}")
    belief = propagate(belief, transition_matrix)
    print(f"After transition:")
    print(format_belief(belief), "\n")
    belief = sense(belief, measurement.value, sensor_matrix)
    print(f"After measurement {measurement.name}:")

    print(format_belief(belief), "\n")
    visualize_belief(belief)


### 1D Histogram Filter with world interaction


In [ ]:
my_env = setup()
num_steps = 10
belief = np.ones(len(my_env.grid)) / len(my_env.grid)
transition_matrix = my_env.get_transition_matrix()
sensor_matrix = my_env.get_sensor_matrix()

for i in range(num_steps):
    if i == 0:
        print("Grid:")
        print(my_env, "\n")

        print(f"Step: {i}")
        print("Initial belief:")
        print(format_belief(belief), "\n")
        visualize_belief(belief)

    print(f"Step: {i + 1}")
    print(f"After transition:")
    my_env.move()
    print(my_env)
    belief = propagate(belief, transition_matrix)
    print(format_belief(belief))

    measurement = my_env.sense()
    print(f"After measurement {measurement.name}:")
    belief = sense(belief, measurement.value, sensor_matrix)
    print(format_belief(belief), "\n")
    visualize_belief(belief)
    print("\n")
